In [1]:
import requests
from bs4 import BeautifulSoup as BS
from itertools import chain
import re
import pandas as pd
from tqdm.notebook import tqdm

In [59]:


cookies = {
    'cf_clearance': 'jJtnI8lqVtOstfwcIvB.Z0qoE3II6i.JL79zlGUYPCI-1736260650-1.2.1.1-FLAxztNG9SntfyWDFiu7p6EaImIpl.c0tsePsvTv_pEwYp_FVZCtZCuV4y1gnYuKZgIT0ULOLYWG2MNSEoXG_EqYQiow_jdBh2sGtfXZGRqz4eyDDW.FffjvDIZLlF8YaTJS8m6IFHYS34AyvaFeq5KxzG0fRNIZnpKlH6BxFQvIisCpVSw9MKJz3RlZscgfWeB7rIxI3b3gK1MRsVYCltUrXbWVz9OsRMt5iB.DCxFoi9zpYoFlMDnmTR8eai6.mKBykrXJry8.NTuvJPpiyKLW27T_G3yfN.RMQeVosL.mY_Tsid3vknCsYJypDMjqA1NjbKZcPneZkHx527BSGV_7q6rqumocXrjykSau6JUdn6iifEq4lDNcqQ.AKohh',
    '_ga_15YCML7VSC': 'GS1.1.1736260651.1.0.1736260651.0.0.0',
    '_ga': 'GA1.1.2143400481.1736260651',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    # 'cookie': 'cf_clearance=jJtnI8lqVtOstfwcIvB.Z0qoE3II6i.JL79zlGUYPCI-1736260650-1.2.1.1-FLAxztNG9SntfyWDFiu7p6EaImIpl.c0tsePsvTv_pEwYp_FVZCtZCuV4y1gnYuKZgIT0ULOLYWG2MNSEoXG_EqYQiow_jdBh2sGtfXZGRqz4eyDDW.FffjvDIZLlF8YaTJS8m6IFHYS34AyvaFeq5KxzG0fRNIZnpKlH6BxFQvIisCpVSw9MKJz3RlZscgfWeB7rIxI3b3gK1MRsVYCltUrXbWVz9OsRMt5iB.DCxFoi9zpYoFlMDnmTR8eai6.mKBykrXJry8.NTuvJPpiyKLW27T_G3yfN.RMQeVosL.mY_Tsid3vknCsYJypDMjqA1NjbKZcPneZkHx527BSGV_7q6rqumocXrjykSau6JUdn6iifEq4lDNcqQ.AKohh; _ga_15YCML7VSC=GS1.1.1736260651.1.0.1736260651.0.0.0; _ga=GA1.1.2143400481.1736260651',
    'priority': 'u=0, i',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

params = {
    'page': '1',
}

response = requests.get('https://novelbin.com/sort/completed', params=params, 
                        # cookies=cookies, headers=headers
                        )

In [14]:
novels_list = []
for page in tqdm(list(range(1,87))):
    params = {
        'page': f'{page}',
    }

    response = requests.get('https://novelbin.com/sort/completed', params=params, 
                            # cookies=cookies, headers=headers
                            )



    soup =  BS(response.text, 'html.parser')
    data_page = (soup
                    .find('main', id='container')
                    .find('div', id='list-page')
                    .find('div', attrs={'class': ['list', 'list-novel']})
                    .find_all('div', attrs={'class': ['row']})
                    )

    row: BS
    for index, row in enumerate(data_page):
        href, title = row.find('h3', attrs={'class': ['novel-title']}).find('a').attrs.values()
        
        markers = (set(chain.from_iterable(
                    map(lambda x: x.attrs.get('class', []), row
                                                            .find('h3', attrs={'class': ['novel-title']})
                                                            .find_next_siblings('span'))
                    )))
        
        new, full, hot = False, False, False
        if 'label-new' in markers:
            new = True
        if 'label-full' in markers:
            full = True
        if 'label-hot' in markers:
            hot = True
        
        text = row.find('div', attrs={'class': ['text-info']}).find('a').get('title')
        pattern = r'^\D*(\d+)'
        try:
            chapters = int(re.findall(pattern, text)[0])
        except IndexError as e:
            print(index, title, text)
            chapters = 0



        novels_list.append((title, new, full, hot, chapters, href))



  0%|          | 0/86 [00:00<?, ?it/s]

0 To Hell With Being a Hero! Afterword ( a year ago )
0 Ze Tian Ji Afterword ( a year ago )
7 Stop, Friendly Fire! afterword ( a year ago )
8 World of Cultivation Nine Hundred and Fifteen – The Ending ( a year ago )
11 Kumo Desu ga, Nani ka? Epilogue ( a year ago )
16 The Charm of Soul Pets Epilogue: She’s in a Certain Space-time ( a year ago )
0 Rebirth of the Thief Who Roamed the World Message From the Translation Team ( a year ago )
2 The Second Coming of Gluttony Afterword ( a year ago )
7 Card Room Wine and Sugar - Extra Eight ( a year ago )


In [16]:
df = pd.DataFrame(novels_list, columns=['title', 'new', 'full', 'hot', 'chapters', 'href'])

In [19]:
df.to_excel('test.xlsx')